<h1>Sommaire<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span><ul class="toc-item"><li><span><a href="#La-métrique-d'évaluation" data-toc-modified-id="La-métrique-d'évaluation-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>La métrique d'évaluation</a></span></li></ul></li><li><span><a href="#Chargement-des-ressources" data-toc-modified-id="Chargement-des-ressources-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Chargement des ressources</a></span><ul class="toc-item"><li><span><a href="#Chargement-du-workspace" data-toc-modified-id="Chargement-du-workspace-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Chargement du workspace</a></span></li></ul></li><li><span><a href="#Développement-des-modèles" data-toc-modified-id="Développement-des-modèles-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Développement des modèles</a></span><ul class="toc-item"><li><span><a href="#Modèle-baseline" data-toc-modified-id="Modèle-baseline-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Modèle baseline</a></span><ul class="toc-item"><li><span><a href="#Recherche-des-hyperparamètres" data-toc-modified-id="Recherche-des-hyperparamètres-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Recherche des hyperparamètres</a></span></li><li><span><a href="#Analyse-des-résultats" data-toc-modified-id="Analyse-des-résultats-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>Analyse des résultats</a></span></li><li><span><a href="#Enregistrement-des-hyperparamètres" data-toc-modified-id="Enregistrement-des-hyperparamètres-3.1.3"><span class="toc-item-num">3.1.3&nbsp;&nbsp;</span>Enregistrement des hyperparamètres</a></span></li></ul></li><li><span><a href="#Modèle-de-type-content-based" data-toc-modified-id="Modèle-de-type-content-based-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Modèle de type content based</a></span><ul class="toc-item"><li><span><a href="#Cold-start-problem" data-toc-modified-id="Cold-start-problem-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Cold start problem</a></span></li><li><span><a href="#Recherche-des-hyperparamètres" data-toc-modified-id="Recherche-des-hyperparamètres-3.2.2"><span class="toc-item-num">3.2.2&nbsp;&nbsp;</span>Recherche des hyperparamètres</a></span></li><li><span><a href="#Analyse-des-résultats" data-toc-modified-id="Analyse-des-résultats-3.2.3"><span class="toc-item-num">3.2.3&nbsp;&nbsp;</span>Analyse des résultats</a></span></li><li><span><a href="#Enregistrement-des-hyperparamètres" data-toc-modified-id="Enregistrement-des-hyperparamètres-3.2.4"><span class="toc-item-num">3.2.4&nbsp;&nbsp;</span>Enregistrement des hyperparamètres</a></span></li></ul></li><li><span><a href="#Modèle-de-collaborative-filtering" data-toc-modified-id="Modèle-de-collaborative-filtering-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Modèle de collaborative filtering</a></span><ul class="toc-item"><li><span><a href="#Cold-start-problem" data-toc-modified-id="Cold-start-problem-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Cold start problem</a></span></li><li><span><a href="#Recherche-des-hyperparamètres" data-toc-modified-id="Recherche-des-hyperparamètres-3.3.2"><span class="toc-item-num">3.3.2&nbsp;&nbsp;</span>Recherche des hyperparamètres</a></span></li><li><span><a href="#Analyse-des-résultats" data-toc-modified-id="Analyse-des-résultats-3.3.3"><span class="toc-item-num">3.3.3&nbsp;&nbsp;</span>Analyse des résultats</a></span></li><li><span><a href="#Enregistrement-des-hyperparamètres" data-toc-modified-id="Enregistrement-des-hyperparamètres-3.3.4"><span class="toc-item-num">3.3.4&nbsp;&nbsp;</span>Enregistrement des hyperparamètres</a></span></li></ul></li></ul></li><li><span><a href="#Sélection-du-meilleur-modèle" data-toc-modified-id="Sélection-du-meilleur-modèle-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Sélection du meilleur modèle</a></span><ul class="toc-item"><li><span><a href="#Comparaison-des-résultats" data-toc-modified-id="Comparaison-des-résultats-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Comparaison des résultats</a></span></li><li><span><a href="#Enregistrement-du-modèle" data-toc-modified-id="Enregistrement-du-modèle-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Enregistrement du modèle</a></span></li></ul></li><li><span><a href="#Analyse-du-modèle" data-toc-modified-id="Analyse-du-modèle-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Analyse du modèle</a></span><ul class="toc-item"><li><span><a href="#Chargement-du-modèle" data-toc-modified-id="Chargement-du-modèle-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Chargement du modèle</a></span></li><li><span><a href="#Chargement-des-données-de-test" data-toc-modified-id="Chargement-des-données-de-test-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Chargement des données de test</a></span></li><li><span><a href="#Evaluation-sur-le-jeu-de-test" data-toc-modified-id="Evaluation-sur-le-jeu-de-test-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Evaluation sur le jeu de test</a></span></li></ul></li><li><span><a href="#Déploiement-manuel-du-MVP" data-toc-modified-id="Déploiement-manuel-du-MVP-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Déploiement manuel du MVP</a></span><ul class="toc-item"><li><span><a href="#Déploiement-du-modèle-sur-Azure-Container-Instances" data-toc-modified-id="Déploiement-du-modèle-sur-Azure-Container-Instances-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Déploiement du modèle sur Azure Container Instances</a></span></li><li><span><a href="#Test-du-modèle" data-toc-modified-id="Test-du-modèle-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Test du modèle</a></span></li><li><span><a href="#Déploiement-du-service-de-prédiction" data-toc-modified-id="Déploiement-du-service-de-prédiction-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Déploiement du service de prédiction</a></span></li><li><span><a href="#Test-du-service-de-prédiction" data-toc-modified-id="Test-du-service-de-prédiction-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Test du service de prédiction</a></span></li><li><span><a href="#Nettoyage-des-ressources" data-toc-modified-id="Nettoyage-des-ressources-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Nettoyage des ressources</a></span></li></ul></li><li><span><a href="#Architecture-de-déploiement-automatisé" data-toc-modified-id="Architecture-de-déploiement-automatisé-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Architecture de déploiement automatisé</a></span></li><li><span><a href="#Pour-aller-plus-loin" data-toc-modified-id="Pour-aller-plus-loin-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Pour aller plus loin</a></span></li></ul></div>

In [ ]:
import sys

%load_ext autoreload
%autoreload 2

sys.path.append("../")
from P9_02_scripts.datasets import *
from P9_02_scripts.models import *
from notebook import *

# Introduction

Ce notebook fait suite au précédent notebook `modélisation.ipynb`. Nous commencerons par tester et évaluer différents modèles de recommandation. Nous allons ensuite les comparer et sélectionner le plus performant. Enfin nous déployerons notre système de recommandation.

Le but de ce projet est de créer au plus vite un MVP qui pourra être présenté à l'équipe et à des utilisteurs. Nous n'allons donc pas rechercher le modèle de recommandation le plus performant.

Nous nous sommes plutôt concentré sur la création et l'implémentation de notre architecture MLOps qui nous a permit d'obtenir rapidement une première version du MVP. Nous pourrons ainsi rapidement itérer sur de futurs versions. De plus, la distribution des données du dataset que nous utilisons ici peut être très différente de celle que nous aurons dans la réalité (articles différents etc...). Cela a été un argument de plus pour nous concentrer sur la mise en place de notre architecture qui nous permettra de rapidement expérimenter et déployer des modèles avec nos futures données.

## La métrique d'évaluation

Notre système de recommandation a pour but de présenter du contenus pertinents à nos utilisateurs. La question est de savoir comment mesurer l'efficacité de ce système.

Il existe plusieurs types de métriques ([en savoir plus](https://github.com/microsoft/recommenders/tree/main/examples/03_evaluate)). Parmis elles, on retrouve :
- `Rating metrics` : Nous allons mesurer la performance du système à prédire les notations des articles (RMSE etc...). Ces métriques sont intrasèques aux modèle et ne sont donc pas adaptés à tous les modèles. De plus, elles sont plus difficilement interprétables car elle ne dépendent pas directement de la pertinance des recommandations faites à l'utilisateur.
- `Ranking metrics` : Permettent de mesurer la pertinance des recommandations pour les utilisateurs (precision, recall etc...). Ces métriques sont des mesure extrinsèques et peuvent être adaptées à tous les modèles. Elle sont facilement interprétables mais peuvent être très couteuse en ressources.
- `Business metrics` : Les précédentes métriques pourraient pousser un modèle à recommander des articles dit "pièges à clicks" et qui ont par exemple des titres très accrocheurs. Cela pourrait faire baisser notre taux de rétention ainsi que le temps passé par les utilisateurs sur notre site. On peux alors utiliser des métriques comme le temps moyen passé par un utilisateur sur notre site et chercher ainsi à l'améliorer.

Les `Business metrics` semblent être les plus intéressantes de notre point de vue. Il nous semble cependant un peu tôt pour en sélectionner une, surtout dans le cadre de ce MVP. Nous allons donc prendre une `Ranking metrics` inspirée du kernel Kaggle suivant : [Recommender Systems in Python 101](https://www.kaggle.com/gspmoreira/recommender-systems-in-python-101#Evaluation). Il s'agit de la métrique `Recall@5` qui va mesurer la proportion de contenu pertinent sur 5 recommandations effectuées parmis un échantillon de 101 articles.

Voici son algorithme :
- Pour chaque article cliqué par un utilisateur :
	- On crée une liste de 100 articles jamais cliqués par cet utilisateur (vrais négatifs).
	- On ajoute l’article cliqué par l’utilisateur dans la liste.
	- On demande au modèle de prédire 5 recommandations parmi ces 101 articles.
	- `user_article_recall` = l’article est dans les 5 recommandations.
- `Recall@5` = moyenne de tous `les user_article_recall`.

Cette algorithme est un bon compromis entre rapidité d'évaluation et interprétabilité. En effet, la partie la plus gourmande en ressources est la création des listes de 100 articles jamais cliqués par les utilisateurs (vrais négatifs). Or cette partie à déjà été réalisée lors de l'ajout de vrais négatifs dans les jeux de notations des articles créés dans le pipeline de transformation des données.

Dans une première version de cet algorithme, nous avons échantillonné 100 articles parmis tous ceux disponibles dans le jeu de données. Dans cette version, on obtenait des scores `Recall@5` proches de 1 simplement en sélectionnant les 5 articles les plus récents. Il s'agissait donc d'un algorithme trop naïf et facilement contournable. L'astuce a été d'échantillonner 100 articles parmis ceux qui ont récemment été cliqués par d'autres utilisateurs et qui ont donc récemment suscité un intérêt. Cela a permis de rendre l'algorithme de calcul du `Recall@5` beaucoup plus robuste.

# Chargement des ressources

Nous allons charger les ressources Azure qui vont nous permettre d'exécuter nos scripts d'entrainement sur des clusters de calcul.

## Chargement du workspace

In [ ]:
# On charge l’espace de travail Azure Machine Learning existant
ws = Workspace.from_config()

# Développement des modèles

Nous allons ici développer et expérimenter divers modèles de recommandation :

<img src="./data/img/archi_dev.png" alt="Développement et expérimentation" style="width:800px;"/>
<p style="text-align: center; text-decoration: underline;">Développement et expérimentation</p>

Dans le précédent notebook, nous avons réalisé les blocs `Data analysis` et `Data transform`. Dans cette section, nous allons voir les blocs `Model training` et `Model evaluation`. Nous allons itérer ces opérations sur divers modèles en recherchant les meilleurs hyperparamètres. Nous allons ensuite sauvegarder les scripts d'entrainement ainsi que les meilleur hyperparamètres de chaque modèle dans notre repository Github.

## Modèle baseline

Nous allons commencer par tester 3 modèles naïfs qui vont nous servir de base de comparaison avec les autres modèles :
1. Recommande les 5 articles les plus récents.
2. Recommande les 5 articles les plus populaires (ceux qui ont le plus de clicks).
3. Recommande 5 articles au hasard.

### Recherche des hyperparamètres

Ces 3 types de recommandation sont implémentés en tant qu'hyperparamètre dans la classe `BaselineRecommender`. Nous allons les tester et ne conserver que la meilleure version pour notre baseline.

Nous allons tester toutes les combinaisons de manière exhaustive en utilisant une recherche par grille. L'entrainement et l'évaluation du modèle étant très rapides, nous n'allons pas utiliser de stratégie d'arrêt anticipée.

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles",
    
    # Hyperparamètres
    "rating_col": DEFAULT_RATING_COL
}

gs_params = {
    "baseline_type": ["most_recent", "most_popular", "random"],
}

In [ ]:
from P9_02_scripts.model_baseline_train.run import exp_submit

# On lance la recherche des hyperparamètres
run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_baseline_train",
    params,
    gs_params=gs_params,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_baseline_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On récupère les hyperparamètres du meilleur modèle
best_hyperparameters = json.loads(run.get_hyperparameters()[best_run.id])
best_hyperparameters

On obtient le meilleur Recall@5 en recommandant 5 articles au hasard.

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
best_run.download_file("outputs/res.parquet", PARQUET_PATH + "model_baseline_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_baseline_train_res.parquet")
res

On obtient un Recall@5 très faible de 0.049 sur le jeu de validation.

Sur toutes les recommandations faites aux utilisateurs, notre système n'a fait que 4.9% de recommandations pertinantes.

### Enregistrement des hyperparamètres

Nous allons enregistrer les meilleurs hyperparamètres dans le fichier `params.json` du dossier contenant les scripts d'entrainement du modèle. Ce fichier sera sauvegardé dans le repository Github et pourra être utilisé lors du déploiement automatisé du modèle.

In [ ]:
# On met en forme les paramètres
best_hyperparameters = {k.replace("--", ""): v for k, v in best_hyperparameters.items()}

In [ ]:
# On met à jour les paramètres de base avec les meilleurs hyperparameters
params.update(best_hyperparameters)

In [ ]:
# On enregistre les paramètres
with open("../P9_02_scripts/model_baseline_train/params.json", "w") as f:
    json.dump(params, f)

## Modèle de type content based

### Cold start problem

### Recherche des hyperparamètres

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles",
    "train_user_profiles": "train_user_profiles",
    
    # Hyperparamètres
    "rating_col": DEFAULT_RATING_COL
}

gs_params = {
    "num_vars_scale": [0., 0.5, 1.],
    "cat_vars_scale": [0., 0.5, 1.]
}

In [ ]:
from P9_02_scripts.model_content_based_train.run import exp_submit

# On lance la recherche des hyperparamètres
run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_content_based_train",
    params,
    gs_params=gs_params,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_content_based_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On récupère les hyperparamètres du meilleur modèle
best_hyperparameters = json.loads(run.get_hyperparameters()[best_run.id])
best_hyperparameters

On obtient le meilleur Recall@5 en utilisant les embeddings ainsi que les catégories des articles dans la recherche de contenus pertinents pour l'utilisateur.

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
best_run.download_file("outputs/res.parquet", PARQUET_PATH + "model_content_based_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_content_based_train_res.parquet")
res

On obtient un Recall@5 de 0.172 sur le jeu de validation.

Sur toutes les recommandations faites aux utilisateurs, notre système a fait 17% de recommandations pertinantes. Ce score est bien meilleur que celui de notre baseline !

### Enregistrement des hyperparamètres

Nous allons enregistrer les meilleurs hyperparamètres dans le fichier `params.json` du dossier contenant les scripts d'entrainement du modèle. Ce fichier sera sauvegardé dans le repository Github et pourra être utilisé lors du déploiement automatisé du modèle.

In [ ]:
# On met en forme les paramètres
best_hyperparameters = {k.replace("--", ""): v for k, v in best_hyperparameters.items()}

In [ ]:
# On met à jour les paramètres de base avec les meilleurs hyperparameters
params.update(best_hyperparameters)

In [ ]:
# On enregistre les paramètres
with open("../P9_02_scripts/model_content_based_train/params.json", "w") as f:
    json.dump(params, f)

## Modèle de collaborative filtering

### Cold start problem

### Recherche des hyperparamètres

In [ ]:
params = {
    # Jeux de données
    "train_user_article_ratings": "train_user_article_ratings",
    "valid_user_article_ratings": "valid_user_article_ratings",
    "article_profiles": "article_profiles"
}

gs_params = {
    "rating_col": ["rating_click_nb", "rating_click_per_session_ratio"],
    "n_factors": [50, 100, 200],
    "n_epochs": [10, 20, 30],
    "lr_all": [0.0001, 0.005, 0.025],
    "reg_all": [0.004, 0.02, 0.1]
}

In [ ]:
from P9_02_scripts.model_collaborative_filtering_train.run import exp_submit

# On lance la recherche des hyperparamètres
run = exp_submit(
    ws,
    SCRIPTS_PATH + "model_collaborative_filtering_train",
    params,
    gs_params=gs_params,
    wait_for_completion=False,
    show_output=False
)

In [ ]:
# # On affiche un widget avec les détails de l'exécution
# RunDetails(run).show()

In [ ]:
# On attend la fin de l'exécution
run.wait_for_completion(show_output=False);

### Analyse des résultats

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_collaborative_filtering_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On récupère les hyperparamètres du meilleur modèle
best_hyperparameters = json.loads(run.get_hyperparameters()[best_run.id])
best_hyperparameters

In [ ]:
# On télécharge le fichier contenant les résultats de l'évaluation du modèle
best_run.download_file("outputs/res.parquet", PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")

In [ ]:
# On ouvre et on affiche les résultats
res = pd.read_parquet(PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")
res

On obtient un Recall@5 très faible de 0.049 sur le jeu de validation.

Sur toutes les recommandations faites aux utilisateurs, notre système n'a fait que 4.9% de recommandations pertinantes. 

Ce modèle ne fait donc pas mieux que la baseline. Cela pourrait être du au fait que ce modèle fasses des recommandations trop variées. Les utilisateurs de ce jeu de données sont peut-être plus intéressés par les mêmes types de contenu, ce qui expliquerait les bien meilleurs résultats obtenus avec le modèle de type content based.

Une autre hypothèse pourrait être que notre système de notation inplicite des articles n'est pas assez représentatif de la pertinance ou non d'un article du point de vu de l'utilisateur.

### Enregistrement des hyperparamètres

Nous allons enregistrer les meilleurs hyperparamètres dans le fichier `params.json` du dossier contenant les scripts d'entrainement du modèle. Ce fichier sera sauvegardé dans le repository Github et pourra être utilisé lors du déploiement automatisé du modèle.

In [ ]:
# On met en forme les paramètres
best_hyperparameters = {k.replace("--", ""): v for k, v in best_hyperparameters.items()}

In [ ]:
# On met à jour les paramètres de base avec les meilleurs hyperparameters
params.update(best_hyperparameters)

In [ ]:
# On enregistre les paramètres
with open("../P9_02_scripts/model_collaborative_filtering_train/params.json", "w") as f:
    json.dump(params, f)

# Sélection du meilleur modèle

## Comparaison des résultats

In [ ]:
model_baseline_train_res = pd.read_parquet(PARQUET_PATH + "model_baseline_train_res.parquet")
model_content_based_train_res = pd.read_parquet(PARQUET_PATH + "model_content_based_train_res.parquet")
model_collaborative_filtering_train_res = pd.read_parquet(PARQUET_PATH + "model_collaborative_filtering_train_res.parquet")

In [ ]:
# On réunit tous les résultats
valid_res = pd.concat([
    model_baseline_train_res,
    model_content_based_train_res,
    model_collaborative_filtering_train_res
])

# On classe les modèles en fonction du meilleur recall obtenu sur le jeu de validation
valid_res = valid_res.sort_values("recall@5", ascending=False).reset_index(drop=True)

# On modifie le nom des colonnes
valid_res = valid_res.rename(columns={
    "precision@5": "valid_precision@5",
    "recall@5": "valid_recall@5",
})
valid_res

On obtient le meilleur Recall@5 sur le jeu de validation avec le modèle de type content based.

## Enregistrement du modèle

Nous allons enregistrer manuellement notre meilleur modèle afin de pouvoir l'évaluer et le déployer dans ce notebook. En production, cette étape sera réalisée automatiquement après l'entrainement du modèle dans le pipeline de déploiement automatisé.

In [ ]:
# On récupère la dernière exécution de l'expérience.
run = get_last_run(ws, "model_content_based_train")

In [ ]:
# On récupère la meilleure exécution de la recherche des hyperparamètres.
best_run = run.get_best_run_by_primary_metric()

In [ ]:
# On enregistre le modèle
model = best_run.register_model(
    model_name="recommender",
    model_path="outputs/model.joblib",
    tags={"class_name": "ContentBasedRecommender"}
)

# Analyse du modèle

Avant de déployer notre modèle, nous allons l'évaluer sur le jeu de test afin de vérifier qu'il généralise correctement sur des données qu'il n'a jamais vu.

## Chargement du modèle

In [ ]:
# On télécharge les données
model = Model(ws, "recommender")
model_path = model.download(target_dir=MODEL_PATH, exist_ok=True)
model_path

In [ ]:
import sys
from P9_02_scripts import models

# On inscrit les modules nécessaires au chargement du modèle
sys.modules['models'] = models

# On charge le modèle
model = joblib.load(model_path)

## Chargement des données de test

In [ ]:
# On charge les datasets
test_user_article_ratings_ds = Dataset.get_by_name(ws, "test_user_article_ratings")
article_profiles_ds = Dataset.get_by_name(ws, "article_profiles")

In [ ]:
# On charge les datasets dans des DataFrames
test_user_article_ratings = test_user_article_ratings_ds.to_pandas_dataframe()
article_profiles = article_profiles_ds.to_pandas_dataframe()

In [ ]:
# On filtre les colonnes pour obtenir le format : user_id, article_id, rating_id
test_ratings = test_user_article_ratings[["user_id", "article_id", DEFAULT_RATING_COL]]
test_ratings = test_ratings.rename(columns={DEFAULT_RATING_COL: "rating"})

## Evaluation sur le jeu de test

In [ ]:
test_res = get_precision_recall_n_score(
    model,
    "ContentBasedRecommender",
    test_ratings,
    article_profiles,
    top_n=5
)

In [ ]:
# On recomme les colonnes
test_res = test_res.rename(columns={
    "precision@5": "test_precision@5",
    "recall@5": "test_recall@5",
})

# On merge les résultats avec ceux du jeu de validation
pd.merge(valid_res, test_res)

On obtient sur le jeu de test un Recall@5 de 0.193.

Ce score est similaire, voir même supérieur à celui obtenu sur le jeu de validation (0.172). Cela semble indiquer que notre modèle généralise correctement.

# Déploiement manuel du MVP

<img src="./data/images/MLOps level 0.svg" alt="MLOps level 0.svg" width="900"/>
<p style="text-align: center; text-decoration: underline;">MLOps level 0 (<a href="https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning">source</a>)</p>

## Déploiement du modèle sur Azure Container Instances

Nous allons utiliser Azure Container Instances. Il s'agit d'une solution simple pour déployer un modèle à titre expérimental. Cette solution est cependant déconseillé par Microsoft pour le déploiement de modèle en production.

Azure va automatiquement créer un servce web et un point de terminaison pour y accéder via une API REST. Quand le service web recevra des tweets à analyser, il les transmettra à un fichier de scoring que l'on doit lui fournir. 

In [ ]:
from P9_02_scripts.model_content_based_deploy_aci.run import model_deploy

model_aci = model_deploy(
    ws,
    SCRIPTS_PATH + "model_content_based_deploy_aci",
    show_output=True
)

In [ ]:
print(model_aci.scoring_uri)

## Test du modèle

In [ ]:
data = {
    "user_id": "1234",
    "session_start_dt": datetime(2017, 10, 16, 12).isoformat(),
    "top_n": 5
}

In [ ]:
r = requests.post(model_aci.scoring_uri, json=data)

if not r.ok:
    print(f"Erreur de type {r.status_code}")

In [ ]:
r.json()

## Déploiement du service de prédiction

In [ ]:
src = SCRIPTS_PATH + "model_content_based_deploy_aci/.env"
dst = FUNCTION_PATH + "recommender/.env"

# On crée le fichier .env dans le dossier de l'azure function
copy2(src, dst)

Création et déploiement de l'azure function :
- `conda activate p9`
- `cd P9_03_function/`
- `./function_create.sh`
- `./function_deploy.sh`

## Test du service de prédiction

In [ ]:
# On lit les configurations
config_parser = ConfigParser()
config_parser.read(FUNCTION_PATH + "function_config.txt")

# On récupère le nom du site
azure_function_name = config_parser.get("DEFAULT", "functionAppName").strip('"')
print(f"Le nom de l'azure function est : {azure_function_name}")

In [ ]:
data = {"userId": "1234"}

In [ ]:
r = requests.post(f"https://{azure_function_name}.azurewebsites.net/api/recommender", json=data)

if not r.ok:
    print(f"Erreur de type {r.status_code}")

In [ ]:
r.json()

## Nettoyage des ressources

On pensera à supprimer le service web à la fin de cette démonstration pour éviter des coût inutiles.

In [ ]:
# On récupère le endpoint du modèle déployé
model_aci = Webservice(ws, "p9-recommender-aci")

# On supprime le endpoint
model_aci.delete()

Suppression du groupe de ressources du service de prédiction
- `conda activate p9`
- `cd P9_03_function/`
- `./function_delete.sh`

# Architecture de déploiement automatisé

<img src="./data/images/MLOps level 1.svg" alt="MLOps level 1.svg" width="900"/>
<p style="text-align: center; text-decoration: underline;">MLOps level 1 (<a href="https://cloud.google.com/architecture/mlops-continuous-delivery-and-automation-pipelines-in-machine-learning">source</a>)</p>

# Pour aller plus loin